In [33]:
targetColName="like"
targetIndex=3
lr=[0.05,0.03,0.07,0.01]
origin="drive/MyDrive/twitter/onechunk/"
eta = 0.1
nrounds = 100
max_depth = 6

#Setup


In [2]:
!pip install cortado

In [3]:
import os   
os.system(f"cp {origin}* ../content/")

0

In [20]:
import cortado as cr
import pandas as pd
train=pd.read_csv("trainFinal.csv", index_col=0)
test=pd.read_csv("testFinal.csv", index_col=0)

columns=["retweet","retweetCom","reply","like"]



for col in columns:
    if col!= targetColName:
        train.drop(col,axis=1,inplace=True)
        test.drop(col,axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
train[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,overlap,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing
0,50,0,0,0,0,2473,662,False,169,339,False,False,0,0,0,0,0,1,0,False,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0
1,39,0,0,1,1,219715,3685,True,629,1473,False,False,1,0,0,0,0,0,1,False,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0
2,40,0,0,0,0,2388283,13511,True,123,200,False,False,0,0,0,0,0,1,0,False,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0
3,66,0,1,0,0,414,720,False,134,379,False,True,1,1,0,0,0,0,1,False,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,673,561,False,2322,996,False,True,0,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [22]:
train = cr.DataFrame.from_pandas(train)
test=cr.DataFrame.from_pandas(test)

In [23]:
train.covariates

[Covariate text_tokens with 2426667 obs: 50.0 39.0 40.0 66.0 3.0 49.0 29.0 44.0 72.0 22.0 ...,
 Covariate hashtags with 2426667 obs: 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...,
 Covariate present_media with 2426667 obs: 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 ...,
 Covariate present_links with 2426667 obs: 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...,
 Covariate present_domains with 2426667 obs: 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...,
 Covariate engaged_with_user_follower_count with 2426667 obs: 2473.0 219715.0 2388283.0 414.0 673.0 9879.0 209681.0 901.0 25956.0 2700.0 ...,
 Covariate engaged_with_user_following_count with 2426667 obs: 662.0 3685.0 13511.0 720.0 561.0 95.0 141.0 782.0 25523.0 2701.0 ...,
 Covariate engaging_user_follower_count with 2426667 obs: 169.0 629.0 123.0 134.0 2322.0 61.0 35.0 9661.0 1120.0 427.0 ...,
 Covariate engaging_user_following_count with 2426667 obs: 339.0 1473.0 200.0 379.0 996.0 1008.0 106.0 8573.0 1959.0 472.0 ...,
 Covariate like with 2426667 

In [25]:
factoredCvTrain=[]
factoredCvTest=[]
for c in train.covariates:
    if c.name!=targetColName:
        factoredCvTrain.append(cr.Factor.from_covariate(train[c.name]))
        factoredCvTest.append(cr.Factor.from_covariate(test[c.name]))

In [26]:
trainTarget=train["like"]

In [27]:
trainTarget

Covariate like with 2426667 obs: 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 ...

In [29]:
factors=train.factors+factoredCvTrain

#Model


In [44]:
trees, pred_cr = cr.xgblogit(trainTarget, factors,  eta = eta, lambda_ = 1.0, gamma = 0.0, minh = 1.0, nrounds =20, maxdepth =6 , slicelen=1000000)

In [45]:
from sklearn.metrics import roc_auc_score
y = trainTarget.to_array() # convert to numpy array
auc_cr = roc_auc_score(y, pred_cr) # cortado auc
print("cortado auc: {auc_cr}".format(auc_cr=auc_cr))

cortado auc: 0.914938316479314


In [46]:
pred_cr

array([0.06551831, 0.56017405, 0.06551478, ..., 0.06551478, 0.73167014,
       0.6397559 ], dtype=float32)

In [47]:
y

array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)

In [49]:
factors[:1]

[Factor engaged_with_user_is_verified with 2426667 obs and 2 levels: False True True False False False False False False False ...]

In [50]:
factor=factors[:1]

In [59]:
t=trees[0][0][0]

In [76]:
t.leftnode.partitions[key]

array([ True,  True,  True])

In [75]:
key=[k for k in t.leftnode.partitions.keys()][0]

In [78]:
for k in  t.leftnode.partitions.keys():
    temp=t.leftnode.partitions[k]
    for i in temp:
        if i==False:
            print(temp)
            print(k)
            break

[False  True False ... False False False]
Factor pagerank_ed_to_ing_ing with 2426667 obs and 186999 levels: [2.9677273e-07,2.9677577e-07) [3.3296106e-07,3.3296317e-07) [-inf,7.998292e-08) [3.568045e-07,3.5680512e-07) [-inf,7.998292e-08) [3.568045e-07,3.5680512e-07) [3.421685e-07,3.421685e-07) [-inf,7.998292e-08) [3.568045e-07,3.5680512e-07) [3.038948e-07,3.0389572e-07) ...


In [87]:
cr.logistic.get_pred(pred_cr)

In [88]:
pred_cr

array([0.6573822 , 0.65878904, 0.65738213, ..., 0.65738213, 0.65922844,
       0.6589967 ], dtype=float32)